In [116]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn
import seaborn as sns
from matplotlib import pyplot as plt
import h5py
import io
from PIL import Image
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
import os

In [117]:
dir = "isic-2024-challenge"
df = pd.read_csv(f"clean-metadata.csv")
df_target1 = df[df['target'] == 1]
df_target0 = df[df['target'] == 0]
df_target0_sampled = df_target0.sample(frac=0.003, random_state=42)
print(len(df_target0_sampled), len(df_target1))
df = pd.concat([df_target1, df_target0_sampled]).reset_index(drop=True)
df = df.sample(frac=1)
train_img = f"{dir}/train-image/image"
train_hdf5 = f"{dir}/train-image.hdf5"
df.head()

1150 383


,Unnamed: 0,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,...,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license,tbp_lv_dnn_lesion_confidence
2,1245,ISIC_0104229,1,IP_9057861,80.0,male,anterior torso,6.55,TBP tile: close-up,3D: white,...,1.539307,3.697858,0.546485,155,84.834820,1441.758000,-60.321040,Memorial Sloan Kettering Cancer Center,CC-BY,66.196170
693,89016,ISIC_2277605,0,IP_9496166,55.0,male,lower extremity,2.63,TBP tile: close-up,3D: XP,...,1.726711,2.139078,0.493151,95,180.902700,340.240700,74.850400,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,98.910330
657,229773,ISIC_5774412,0,IP_0680005,20.0,male,posterior torso,2.62,TBP tile: close-up,3D: XP,...,0.871090,2.209343,0.369369,125,25.009888,971.638489,179.198608,"Department of Dermatology, Hospital Clínic de ...",CC-BY-NC,73.640847
648,180921,ISIC_4558822,0,IP_3220951,60.0,male,posterior torso,3.79,TBP tile: close-up,3D: white,...,2.147844,2.616493,0.370130,145,-146.598618,1341.177246,136.413635,Memorial Sloan Kettering Cancer Center,CC-BY,99.991775
367,379199,ISIC_9454701,1,IP_7696347,70.0,male,anterior torso,6.44,TBP tile: close-up,3D: XP,...,8.246979,2.034377,0.143511,10,-118.412000,1315.588000,6.993042,"Department of Dermatology, Hospital Clínic de ...",CC-BY-NC,100.000000


In [118]:
numeric_cols = [
    "age_approx",
    "clin_size_long_diam_mm",
    "tbp_lv_A",
    "tbp_lv_Aext",
    "tbp_lv_B",
    "tbp_lv_Bext",
    "tbp_lv_C",
    "tbp_lv_Cext",
    "tbp_lv_H",
    "tbp_lv_Hext",
    "tbp_lv_L",
    "tbp_lv_Lext",
    "tbp_lv_areaMM2",
    "tbp_lv_area_perim_ratio",
    "tbp_lv_color_std_mean",
    "tbp_lv_deltaA",
    "tbp_lv_deltaB",
    "tbp_lv_deltaL",
    "tbp_lv_deltaLB",
    "tbp_lv_deltaLBnorm",
    "tbp_lv_eccentricity",
    "tbp_lv_minorAxisMM",
    "tbp_lv_nevi_confidence",
    "tbp_lv_norm_border",
    "tbp_lv_norm_color",
    "tbp_lv_perimeterMM",
    "tbp_lv_radial_color_std_max",
    "tbp_lv_stdL",
    "tbp_lv_stdLExt",
    "tbp_lv_symm_2axis",
    "tbp_lv_symm_2axis_angle",
    "tbp_lv_x",
    "tbp_lv_y",
    "tbp_lv_z",
    "tbp_lv_dnn_lesion_confidence",
]
categoric_cols = ['sex', 'anatom_site_general', "tbp_lv_location", "tbp_lv_location_simple"]

df_categorical = pd.get_dummies(df[categoric_cols], drop_first=True)
df_numeric = df[numeric_cols].copy()

scaler = StandardScaler()
df_numeric[numeric_cols] = scaler.fit_transform(df_numeric[numeric_cols])

processed = pd.concat([df_numeric, df_categorical], axis=1)

clean_df = df.copy()
for col in processed.columns:
    clean_df[col] = processed[col]

In [119]:
def decode_isic_image(isic_id, file_path, num_channels=3, as_array=False):
    """
    Decodes an ISIC image from an HDF5 file.

    The HDF5 file is expected to store images with keys corresponding to their ISIC IDs.
    The image can be stored either as encoded bytes (JPEG, PNG, etc.) or as a raw NumPy array.

    Parameters:
        isic_id (str): The ISIC identifier referencing the image in the HDF5 file.
        file_path (str): The path to the HDF5 file.
        num_channels (int): The expected number of channels in the image. For example, 1 for grayscale,
                            3 for RGB, or 4 for RGBA.
        as_array (bool): If True, returns a NumPy array; otherwise returns a PIL Image (default).

    Returns:
        PIL.Image.Image or numpy.ndarray: The decoded image.

    Raises:
        ValueError: If the image cannot be found or decoded, or if the image channels do not
                    match the expected number.
    """
    # Open the HDF5 file and retrieve the image using its ISIC ID.
    with h5py.File(file_path, "r") as hf:
        try:
            # Adjust this line if the images are stored under a subgroup (e.g., hf['images'][isic_id]).
            data = hf[isic_id][()]
        except KeyError:
            raise ValueError(f"Image with ISIC ID '{isic_id}' not found in the HDF5 file.")

    # Case 1: The data is stored as encoded image bytes.
    if isinstance(data, bytes):
        image = Image.open(io.BytesIO(data))

    # Case 2: The data is stored as a raw NumPy array.
    elif isinstance(data, np.ndarray):
        # If the image is grayscale (2D array) and one channel is expected:
        if data.ndim == 2 and num_channels == 1:
            image = Image.fromarray(data, mode='L')
        # For color images, we expect a 3D array.
        elif data.ndim == 3:
            if data.shape[2] == num_channels:
                if num_channels == 1:
                    # Squeeze the extra dimension for grayscale.
                    image = Image.fromarray(data.squeeze(), mode='L')
                elif num_channels == 3:
                    image = Image.fromarray(data, mode='RGB')
                elif num_channels == 4:
                    image = Image.fromarray(data, mode='RGBA')
                else:
                    # For uncommon channel counts, fall back to default conversion.
                    image = Image.fromarray(data)
            else:
                raise ValueError(f"Expected {num_channels} channels, but found {data.shape[2]} channels in the image data.")
        else:
            raise ValueError("Unsupported image data shape in the HDF5 file.")

    # Case 3: Attempt to convert any other type to bytes and decode.
    else:
        try:
            data_bytes = bytes(data)
            image = Image.open(io.BytesIO(data_bytes))
        except Exception as e:
            raise ValueError("Could not decode image from the HDF5 file data.") from e

    if as_array:
        return np.array(image)
    else:
        return image

In [120]:
class SkinLesionData(Dataset):
    def __init__(self, csv_file, img_dir, hdf5_path, transform=None):
        self.df = csv_file
        self.img_dir = img_dir
        self.transform = transform
        self.hdf5_path = hdf5_path
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        isic_id = row["isic_id"]
        label = int(row["target"])
        
        image = decode_isic_image(isic_id, self.hdf5_path, num_channels=3, as_array=False)

        if self.transform:
            image = self.transform(image)
        
        return image, label

In [121]:
import torchvision.transforms as T

train_transforms = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),  # Random flip
    T.RandomRotation(15),       # Random rotation
    T.ColorJitter(brightness=0.1, contrast=0.1),  # Color variation
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [122]:
from torch.utils.data import DataLoader, random_split

dataset = SkinLesionData(csv_file=df, img_dir=train_img, hdf5_path=train_hdf5, transform=train_transforms)
train_size = int(0.2 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)


In [123]:
import torchvision.models as models

model = models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(nn.Linear(num_ftrs, 1))

model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class_counts = torch.tensor([len(df_target0_sampled), len(df_target1)])
class_weights = 1. / class_counts
class_weights = class_weights / class_weights.sum() 
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([class_weights[1]/class_weights[0]]).to(device))

optimizer = optim.Adam(model.parameters(), lr=5e-5)

num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Forward
        outputs = model(images)       # shape: (batch_size, 2)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Accumulate training metrics
        running_loss += loss.item() * images.size(0)
        predicted = (outputs > 0.5).float().squeeze()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    train_loss = running_loss / total
    train_acc = correct / total
    
    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            val_loss += loss.item() * images.size(0)
            
            predicted = (outputs > 0.5).float().squeeze()
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)
    
    val_loss /= val_total
    val_acc = val_correct / val_total
    
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


Epoch [1/30] Train Loss: 0.1726, Train Acc: 0.9608, Val Loss: 0.8229, Val Acc: 0.8802
